In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras import datasets ,layers,models
import matplotlib.pyplot as plt
from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications import *
from keras.callbacks import ReduceLROnPlateau

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

In [ ]:
num_classes = 1
input_shape = (1000, 1000, 3)


In [ ]:
img = image.load_img("../input/chest-xray-pneumonia/chest_xray/test/NORMAL/IM-0001-0001.jpeg")
img

In [ ]:
plt.imshow(img)


In [ ]:

import cv2
img_num = cv2.imread("../input/chest-xray-pneumonia/chest_xray/test/NORMAL/IM-0001-0001.jpeg")
img_num.shape

In [ ]:
img_num

In [ ]:
### train = ImageDataGenerator(rescale = 1/255)
train = ImageDataGenerator(rescale=1./255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images



valid = ImageDataGenerator(rescale=1./255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images 





In [ ]:
train_dataset = train.flow_from_directory(
        '../input/chest-xray-pneumonia/chest_xray/train/',
        target_size=(1000, 1000),  
        class_mode='binary')
validation_dataset = valid.flow_from_directory(
        '../input/chest-xray-pneumonia/chest_xray/val/',
        target_size=(1000, 1000),
        class_mode='binary')

In [ ]:
train_dataset.class_indices
img_size =1000

#    **load pre-trained EfficientNetB6 baseline model with the imagNet Dataset **

In [ ]:

conv_base = EfficientNetB6(input_shape=(img_size,img_size,3),include_top=False,weights="imagenet")


# # **base-line Model**

In [ ]:
cnn4 = Sequential()
cnn4.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224,224,3)))
cnn4.add(BatchNormalization())

cnn4.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
cnn4.add(Dropout(0.25))

cnn4.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.25))

cnn4.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(MaxPooling2D(pool_size=(2, 2)))
cnn4.add(Dropout(0.25))

cnn4.add(Flatten())

cnn4.add(Dense(512, activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.5))

cnn4.add(Dense(128, activation='relu'))
cnn4.add(BatchNormalization())
cnn4.add(Dropout(0.5))

cnn4.add(Dense(1, activation='sigmoid'))


cnn4.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience = 2,
                                            verbose=1,
                                            factor=0.1,
                                            min_lr=0.000001)
cnn4.compile(optimizer = opt, loss='binary_crossentropy', metrics=['accuracy'])




In [ ]:
history = cnn4.fit(train_dataset, validation_data=validation_dataset, epochs = 15, callbacks = [learning_rate_reduction])
history

In [ ]:
test_dataset = valid.flow_from_directory(
        '../input/chest-xray-pneumonia/chest_xray/test/',
        target_size=(224, 224),
        class_mode='binary')

In [ ]:
print("Evaluate on test data")
results = cnn4.evaluate(test_dataset)
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")


# # **VGG16 Model**

In [ ]:
# Create the base model of VGG19
vgg19 = VGG19( include_top=False, input_shape = (224, 224, 3) , weights = 'imagenet')


In [ ]:
vgg19.summary()

###### 

In [ ]:
model = Sequential()

for layer in vgg16.layers[:-1]: # this is where I changed your code
    model.add(layer)    
# Freeze the layers 
#for layer in model.layers:
#    layer.trainable = False


In [ ]:
model.add(Flatten())
model.add(Dense(64 , activation = 'relu'))
model.add(Dense(32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer = opt, loss='binary_crossentropy', metrics=[tf.keras.metrics.BinaryCrossentropy()])


In [ ]:
history = model.fit(train_dataset, validation_data=validation_dataset, epochs = 5, batch_size = 64)
history

In [ ]:
test_dataset = valid.flow_from_directory(
        '../input/chest-xray-pneumonia/chest_xray/test/',
        target_size=(224,224),
        class_mode='binary')

In [ ]:
print("Evaluate on test data")
results = model.evaluate(test_dataset)
print("test loss, test acc:", results)
print("Accuracy of the model is - " , results[1]*100 , "%")


In [ ]:
from matplotlib import pyplot

# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

# VIT model 

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 64
num_epochs = 100
image_size = 1000  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 4
mlp_head_units = [64, 32]  # Size of the dense layers of the final classifier

# # Implement multilayer perceptron (MLP)


In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

# # Implement patch creation as a layer


In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

# # Implement the patch encoding layer
The PatchEncoder layer will linearly transform a patch by projecting it into a vector of size projection_dim. In addition, it adds a learnable position embedding to the projected vector.

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

# # Build the ViT model


In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(1, activation='sigmoid')(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    
    return model

# # Compile, train, and evaluate the mode


In [ ]:
def run_experiment(model):
    opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

    model.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=[ 'accuracy' ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(train_dataset, validation_data=validation_dataset, epochs = 5, batch_size = 64,callbacks=[checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    return history


vit_classifier = create_vit_classifier()
print(vit_classifier.summary())


In [ ]:
history = run_experiment(vit_classifier)